##### Libraries

In [ ]:
import numpy as np  # for numerical operations.

import pandas as pd  # for manipulation and analysis.

import seaborn as sns # for visualization.

import matplotlib.pyplot as plt # for visualization.

from sklearn.preprocessing import LabelEncoder # for encoding categorical variables.

from sklearn.preprocessing import StandardScaler # for standardizing numerical variables.

from sklearn.impute import SimpleImputer # for imputing missing values.

from sklearn.pipeline import make_pipeline # for creating pipelines.

from sklearn.linear_model import LogisticRegression # for logistic regression.

from sklearn.model_selection import train_test_split # for splitting data into training and testing sets.

from sklearn.ensemble import RandomForestClassifier # for random forest classifier.

from sklearn.metrics import classification_report, roc_auc_score # for evaluating model performance.

from xgboost import XGBClassifier # for XGBoost classifier.

from sklearn.model_selection import StratifiedKFold, cross_val_score # for cross-validation.

from sklearn.metrics import make_scorer, f1_score # for calculating F1 score.

import lightgbm as lgb # for LightGBM classifier.

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # for confusion matrix.

from sklearn.metrics import precision_recall_curve, average_precision_score # for precision-recall curve.

##### Loading datasets

In [ ]:
import kagglehub
import os

# Download the latest version of the dataset
path = kagglehub.dataset_download("rohitrox/healthcare-provider-fraud-detection-analysis")

# Print the dataset path
print("Path to dataset files:", path)

# List files inside the dataset directory
print("\nFiles in the dataset:")
for file in os.listdir(path):
    print("-", file)

Path to dataset files: C:\Users\Administrator\.cache\kagglehub\datasets\rohitrox\healthcare-provider-fraud-detection-analysis\versions\1

Files in the dataset:
- Test-1542969243754.csv
- Test_Beneficiarydata-1542969243754.csv
- Test_Inpatientdata-1542969243754.csv
- Test_Outpatientdata-1542969243754.csv
- Train-1542865627584.csv
- Train_Beneficiarydata-1542865627584.csv
- Train_Inpatientdata-1542865627584.csv
- Train_Outpatientdata-1542865627584.csv


In [ ]:
# Set dataset path
dataset_path = r"C:\Users\Administrator\.cache\kagglehub\datasets\rohitrox\healthcare-provider-fraud-detection-analysis\versions\1"

# Load Train Dataset
Train_Provider = pd.read_csv(os.path.join(dataset_path, 'Train-1542865627584.csv'))
Train_Beneficiarydata = pd.read_csv(os.path.join(dataset_path, 'Train_Beneficiarydata-1542865627584.csv'))
Train_Inpatientdata = pd.read_csv(os.path.join(dataset_path, 'Train_Inpatientdata-1542865627584.csv'))
Train_Outpatientdata = pd.read_csv(os.path.join(dataset_path, 'Train_Outpatientdata-1542865627584.csv'))

# Load Test Dataset
Test_Provider = pd.read_csv(os.path.join(dataset_path, 'Test-1542969243754.csv'))
Test_Beneficiarydata = pd.read_csv(os.path.join(dataset_path, 'Test_Beneficiarydata-1542969243754.csv'))
Test_Inpatientdata = pd.read_csv(os.path.join(dataset_path, 'Test_Inpatientdata-1542969243754.csv'))
Test_Outpatientdata = pd.read_csv(os.path.join(dataset_path, 'Test_Outpatientdata-1542969243754.csv'))

##### Shape of the datasets

In [ ]:
def print_data_shapes(*dataframes, title="Data Shapes"):
    print("="*40)
    print(title.center(40))
    print("="*40)
    for df_name, df in dataframes:
        print(f"{df_name:<25} : {df.shape}")
    print("="*40)

# Before merging
print_data_shapes(
    ("Train_Provider", Train_Provider),
    ("Train_Beneficiarydata", Train_Beneficiarydata),
    ("Train_Inpatientdata", Train_Inpatientdata),
    ("Train_Outpatientdata", Train_Outpatientdata),
    title="Train Data Shapes"
)

print_data_shapes(
    ("Test_Provider", Test_Provider),
    ("Test_Beneficiarydata", Test_Beneficiarydata),
    ("Test_Inpatientdata", Test_Inpatientdata),
    ("Test_Outpatientdata", Test_Outpatientdata),
    title="Test Data Shapes"
)

           Train Data Shapes            
Train_Provider            : (5410, 2)
Train_Beneficiarydata     : (138556, 25)
Train_Inpatientdata       : (40474, 30)
Train_Outpatientdata      : (517737, 27)
            Test Data Shapes            
Test_Provider             : (1353, 1)
Test_Beneficiarydata      : (63968, 25)
Test_Inpatientdata        : (9551, 30)
Test_Outpatientdata       : (125841, 27)


In [ ]:
def merge_and_return_shapes(train_inpatient, test_inpatient,
                            train_outpatient, test_outpatient,
                            train_beneficiary, test_beneficiary,
                            provider_data=None):
    # Merge Inpatient with Beneficiary
    train_inpatient_merged = pd.merge(train_inpatient, train_beneficiary, on='BeneID', how='left')
    test_inpatient_merged = pd.merge(test_inpatient, test_beneficiary, on='BeneID', how='left')
    
    # Merge Outpatient with Beneficiary
    train_outpatient_merged = pd.merge(train_outpatient, train_beneficiary, on='BeneID', how='left')
    test_outpatient_merged = pd.merge(test_outpatient, test_beneficiary, on='BeneID', how='left')
    
    # Combine Inpatient and Outpatient for Train and Test separately
    train_combined = pd.concat([train_inpatient_merged, train_outpatient_merged], axis=0, ignore_index=True)
    test_combined = pd.concat([test_inpatient_merged, test_outpatient_merged], axis=0, ignore_index=True)
    
    # Optionally merge with Provider data (only applicable for Train)
    if provider_data is not None:
        train_combined = pd.merge(train_combined, provider_data, on='Provider', how='left')

    # Return the merged datasets
    return train_combined, test_combined


# call the function to merge and get the combined datasets:
train_combined, test_combined = merge_and_return_shapes(
    Train_Inpatientdata, Test_Inpatientdata, 
    Train_Outpatientdata, Test_Outpatientdata,
    Train_Beneficiarydata, Test_Beneficiarydata,
    provider_data=Train_Provider
)

# Print shapes of combined datasets after merging
print_data_shapes(
    ("Train Combined", train_combined),
    ("Test Combined", test_combined),
    title="Combined Data Shapes After Merging"
)

   Combined Data Shapes After Merging   
Train Combined            : (558211, 55)
Test Combined             : (135392, 54)


##### .info()

In [ ]:
def missing_values_summary(df):
    import pandas as pd

    total = len(df)

    # summary df
    missing_df = df.isnull().sum().reset_index()
    missing_df.columns = ['Column', 'Missing_Count']
    missing_df['Missing_Percent'] = (missing_df['Missing_Count'] / total * 100).round(2)
    missing_df['Dtype'] = df.dtypes.values
    missing_df['Total'] = total

    # filter only columns with missing values
    missing_df = missing_df[missing_df['Missing_Count'] > 0]

    if missing_df.empty:
        print("✅ No missing values found in the dataset.")
        return

    # sort by dtype and missing percentage
    missing_df = missing_df.sort_values(by=['Dtype', 'Missing_Percent'], ascending=[True, False])

    # group and print
    for dtype, group in missing_df.groupby('Dtype'):
        print("=" * 70)
        print(f"Missing Values Summary for Data Type: {dtype}".center(70))
        print("=" * 70)
        display(group[['Column', 'Missing_Count', 'Missing_Percent', 'Total']].reset_index(drop=True))

In [ ]:
# Missing entries for train dataset
missing_values_summary(train_combined)

            Missing Values Summary for Data Type: float64             


,Column,Missing_Count,Missing_Percent,Total
0,ClmProcedureCode_5,558202,100.00,558211
1,ClmProcedureCode_6,558211,100.00,558211
2,ClmProcedureCode_4,558093,99.98,558211
3,ClmProcedureCode_3,557242,99.83,558211
4,ClmProcedureCode_2,552721,99.02,558211
5,ClmProcedureCode_1,534901,95.82,558211
6,DeductibleAmtPaid,899,0.16,558211


             Missing Values Summary for Data Type: object             


,Column,Missing_Count,Missing_Percent,Total
0,DOD,554080,99.26,558211
1,ClmDiagnosisCode_10,553201,99.10,558211
2,AdmissionDt,517737,92.75,558211
3,DischargeDt,517737,92.75,558211
4,DiagnosisGroupCode,517737,92.75,558211
5,ClmDiagnosisCode_9,516396,92.51,558211
6,ClmDiagnosisCode_8,504767,90.43,558211
7,ClmDiagnosisCode_7,492034,88.14,558211
8,ClmDiagnosisCode_6,473819,84.88,558211
9,ClmDiagnosisCode_5,446287,79.95,558211


In [ ]:
# Missing entries for test dataset
missing_values_summary(test_combined)

            Missing Values Summary for Data Type: float64             


,Column,Missing_Count,Missing_Percent,Total
0,ClmProcedureCode_5,135390,100.00,135392
1,ClmProcedureCode_6,135392,100.00,135392
2,ClmProcedureCode_4,135363,99.98,135392
3,ClmProcedureCode_3,135167,99.83,135392
4,ClmProcedureCode_2,134129,99.07,135392
5,ClmProcedureCode_1,129925,95.96,135392
6,DeductibleAmtPaid,196,0.14,135392


             Missing Values Summary for Data Type: object             


,Column,Missing_Count,Missing_Percent,Total
0,DOD,134352,99.23,135392
1,ClmDiagnosisCode_10,134242,99.15,135392
2,AdmissionDt,125841,92.95,135392
3,DischargeDt,125841,92.95,135392
4,DiagnosisGroupCode,125841,92.95,135392
5,ClmDiagnosisCode_9,125516,92.71,135392
6,ClmDiagnosisCode_8,122670,90.60,135392
7,ClmDiagnosisCode_7,119607,88.34,135392
8,ClmDiagnosisCode_6,115232,85.11,135392
9,ClmDiagnosisCode_5,108594,80.21,135392


##### Dropping all columns with .90 threshold.

In [ ]:
import pandas as pd

def clean_high_missing_columns(df, threshold=90):
    RED = '\033[91m'
    YELLOW = '\033[93m'
    GREEN = '\033[92m'
    RESET = '\033[0m'

    total = len(df)
    missing_df = df.isnull().sum().reset_index()
    missing_df.columns = ['Column', 'Missing_Count']
    missing_df['Missing_Percent'] = (missing_df['Missing_Count'] / total * 100).round(2)
    missing_df['Dtype'] = df.dtypes.values
    missing_df['Total'] = total

    # Columns to drop
    to_drop = missing_df[missing_df['Missing_Percent'] >= threshold]

    if not to_drop.empty:
        print("=" * 70)
        print(f"{RED}[!] Dropping columns with ≥ {threshold}% missing values".center(70) + RESET)
        print("=" * 70)
        display(to_drop[['Column', 'Missing_Count', 'Missing_Percent', 'Total']])
        df = df.drop(columns=to_drop['Column'].values)
    else:
        print(f"{GREEN}[✓] No columns with ≥ {threshold}% missing values found.{RESET}")

    # Drop rows where 'DeductibleAmtPaid' is missing
    if 'DeductibleAmtPaid' in df.columns:
        missing_rows = df[df['DeductibleAmtPaid'].isnull()]
        if not missing_rows.empty:
            print(f"{RED}[!] Dropping {len(missing_rows)} rows where 'DeductibleAmtPaid' is missing{RESET}")
            df = df.dropna(subset=['DeductibleAmtPaid'])

    # Replace missing values in diagnosis code columns with 'Not Assigned'
    diagnosis_columns = [col for col in df.columns if 'ClmDiagnosisCode' in col or col == 'ClmAdmitDiagnosisCode']
    for column in diagnosis_columns:
        if column in df.columns:
            missing_count = df[column].isnull().sum()
            if missing_count > 0:
                print(f"{YELLOW}[~] Filling {missing_count} missing values in '{column}' with 'Not Assigned'{RESET}")
                df[column] = df[column].fillna('Not Assigned')

    # Replace missing values in physician columns with 'Not Assigned'
    physician_columns = ['OperatingPhysician', 'AttendingPhysician', 'OtherPhysician']
    for column in physician_columns:
        if column in df.columns:
            missing_count = df[column].isnull().sum()
            if missing_count > 0:
                print(f"{YELLOW}[~] Filling {missing_count} missing values in '{column}' with 'Not Assigned'{RESET}")
                df[column] = df[column].fillna('Not Assigned')

    return df

In [ ]:
# clean up missing values, threshold 0.9
train_combined = clean_high_missing_columns(train_combined)

         [!] Dropping columns with ≥ 90% missing values          


,Column,Missing_Count,Missing_Percent,Total
9,AdmissionDt,517737,92.75,558211
12,DischargeDt,517737,92.75,558211
13,DiagnosisGroupCode,517737,92.75,558211
21,ClmDiagnosisCode_8,504767,90.43,558211
22,ClmDiagnosisCode_9,516396,92.51,558211
23,ClmDiagnosisCode_10,553201,99.10,558211
24,ClmProcedureCode_1,534901,95.82,558211
25,ClmProcedureCode_2,552721,99.02,558211
26,ClmProcedureCode_3,557242,99.83,558211
27,ClmProcedureCode_4,558093,99.98,558211


[!] Dropping 899 rows where 'DeductibleAmtPaid' is missing
[~] Filling 412312 missing values in 'ClmAdmitDiagnosisCode' with 'Not Assigned'
[~] Filling 10453 missing values in 'ClmDiagnosisCode_1' with 'Not Assigned'
[~] Filling 195601 missing values in 'ClmDiagnosisCode_2' with 'Not Assigned'
[~] Filling 315143 missing values in 'ClmDiagnosisCode_3' with 'Not Assigned'
[~] Filling 393650 missing values in 'ClmDiagnosisCode_4' with 'Not Assigned'
[~] Filling 446238 missing values in 'ClmDiagnosisCode_5' with 'Not Assigned'
[~] Filling 473744 missing values in 'ClmDiagnosisCode_6' with 'Not Assigned'
[~] Filling 491921 missing values in 'ClmDiagnosisCode_7' with 'Not Assigned'
[~] Filling 443409 missing values in 'OperatingPhysician' with 'Not Assigned'
[~] Filling 1499 missing values in 'AttendingPhysician' with 'Not Assigned'
[~] Filling 357674 missing values in 'OtherPhysician' with 'Not Assigned'


In [ ]:
# clean up missing values, threshold 0.9
test_combined = clean_high_missing_columns(test_combined)

         [!] Dropping columns with ≥ 90% missing values          


,Column,Missing_Count,Missing_Percent,Total
9,AdmissionDt,125841,92.95,135392
12,DischargeDt,125841,92.95,135392
13,DiagnosisGroupCode,125841,92.95,135392
21,ClmDiagnosisCode_8,122670,90.60,135392
22,ClmDiagnosisCode_9,125516,92.71,135392
23,ClmDiagnosisCode_10,134242,99.15,135392
24,ClmProcedureCode_1,129925,95.96,135392
25,ClmProcedureCode_2,134129,99.07,135392
26,ClmProcedureCode_3,135167,99.83,135392
27,ClmProcedureCode_4,135363,99.98,135392


[!] Dropping 196 rows where 'DeductibleAmtPaid' is missing
[~] Filling 100036 missing values in 'ClmAdmitDiagnosisCode' with 'Not Assigned'
[~] Filling 2578 missing values in 'ClmDiagnosisCode_1' with 'Not Assigned'
[~] Filling 47783 missing values in 'ClmDiagnosisCode_2' with 'Not Assigned'
[~] Filling 76740 missing values in 'ClmDiagnosisCode_3' with 'Not Assigned'
[~] Filling 95770 missing values in 'ClmDiagnosisCode_4' with 'Not Assigned'
[~] Filling 108586 missing values in 'ClmDiagnosisCode_5' with 'Not Assigned'
[~] Filling 115217 missing values in 'ClmDiagnosisCode_6' with 'Not Assigned'
[~] Filling 119585 missing values in 'ClmDiagnosisCode_7' with 'Not Assigned'
[~] Filling 108132 missing values in 'OperatingPhysician' with 'Not Assigned'
[~] Filling 345 missing values in 'AttendingPhysician' with 'Not Assigned'
[~] Filling 86587 missing values in 'OtherPhysician' with 'Not Assigned'


In [ ]:
# Missing entries for train dataset, after clean_up
missing_values_summary(train_combined)

✅ No missing values found in the dataset.


In [ ]:
# Missing entries for test dataset, after clean_up
missing_values_summary(test_combined)

✅ No missing values found in the dataset.


In [ ]:
import pandas as pd

def check_duplicates(train_combined, test_combined):
    print("~" * 70)
    print("🔍 Checking for duplicate rows in each dataset".center(70))
    print("~" * 70)

    # Train dataset
    train_duplicates = train_combined[train_combined.duplicated()]
    if not train_duplicates.empty:
        print(f"[!] Train set has {len(train_duplicates)} duplicate rows.")
    else:
        print("[✓] No duplicates found in train set.")

    # Test dataset
    test_duplicates = test_combined[test_combined.duplicated()]
    if not test_duplicates.empty:
        print(f"[!] Test set has {len(test_duplicates)} duplicate rows.")
    else:
        print("[✓] No duplicates found in test set.")

    return train_duplicates, test_duplicates

In [ ]:
# duplicates check
train_duplicates, test_duplicates = check_duplicates(train_combined, test_combined)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            🔍 Checking for duplicate rows in each dataset             
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[✓] No duplicates found in train set.
[✓] No duplicates found in test set.


In [ ]:
def convert_date_columns(df, date_columns):
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            print(f"[✓] Converted '{col}' to datetime")
        else:
            print(f"[!] Column '{col}' not found in DataFrame")
    return df

In [ ]:
date_cols = ['ClaimStartDt', 'ClaimEndDt', 'DOB']

# Convert in both datasets
train_combined = convert_date_columns(train_combined, date_cols)
test_combined = convert_date_columns(test_combined, date_cols)

[✓] Converted 'ClaimStartDt' to datetime
[✓] Converted 'ClaimEndDt' to datetime
[✓] Converted 'DOB' to datetime
[✓] Converted 'ClaimStartDt' to datetime
[✓] Converted 'ClaimEndDt' to datetime
[✓] Converted 'DOB' to datetime


In [ ]:
def check_date_format(df, date_columns, df_name='Dataset'):
    print(f"\n{'~'*70}")
    print(f"🗓️  Date Format Check in {df_name}")
    print(f"{'~'*70}")
    for col in date_columns:
        if col in df.columns:
            dtype = df[col].dtype
            is_datetime = pd.api.types.is_datetime64_any_dtype(df[col])
            status = "✅ datetime64[ns]" if is_datetime else f"❌ {dtype}"
            print(f"{col:<20} : {status}")
        else:
            print(f"{col:<20} : ⚠️ Column not found")

In [ ]:
date_cols = ['ClaimStartDt', 'ClaimEndDt', 'DOB']

check_date_format(train_combined, date_cols, 'Train Dataset')
check_date_format(test_combined, date_cols, 'Test Dataset')


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
🗓️  Date Format Check in Train Dataset
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ClaimStartDt         : ✅ datetime64[ns]
ClaimEndDt           : ✅ datetime64[ns]
DOB                  : ✅ datetime64[ns]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
🗓️  Date Format Check in Test Dataset
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ClaimStartDt         : ✅ datetime64[ns]
ClaimEndDt           : ✅ datetime64[ns]
DOB                  : ✅ datetime64[ns]


In [ ]:
# Chronic Conditions Count
train_combined['ClaimDuration'] = (pd.to_datetime(train_combined['ClaimEndDt']) - pd.to_datetime(train_combined['ClaimStartDt'])).dt.days
test_combined['ClaimDuration'] = (pd.to_datetime(test_combined['ClaimEndDt']) - pd.to_datetime(test_combined['ClaimStartDt'])).dt.days

In [ ]:
# latest claim date as the reference date
latest_claim_date = pd.to_datetime(train_combined['ClaimEndDt']).max()

# Calculate age based on the latest claim date (or any other suitable date)
train_combined['Age'] = (latest_claim_date - pd.to_datetime(train_combined['DOB'])).dt.days // 365
test_combined['Age'] = (latest_claim_date - pd.to_datetime(test_combined['DOB'])).dt.days // 365

In [ ]:
chronic_conditions_columns = [
    'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
    'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
    'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
    'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke'
]
train_combined['TotalChronicConditions'] = train_combined[chronic_conditions_columns].sum(axis=1)
test_combined['TotalChronicConditions'] = test_combined[chronic_conditions_columns].sum(axis=1)

In [ ]:
# Reimbursement vs Deductible Ratio
train_combined['ReimbursementDeductibleRatio'] = train_combined['InscClaimAmtReimbursed'] / train_combined['DeductibleAmtPaid']
test_combined['ReimbursementDeductibleRatio'] = test_combined['InscClaimAmtReimbursed'] / test_combined['DeductibleAmtPaid']

In [ ]:
# claim types
train_combined['ClaimType'] = train_combined[['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3']].apply(lambda x: '-'.join(x.dropna().astype(str)), axis=1)
test_combined['ClaimType'] = test_combined[['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3']].apply(lambda x: '-'.join(x.dropna().astype(str)), axis=1)

In [ ]:
# fraud indicator
train_combined['IsFraud'] = train_combined['PotentialFraud'].apply(lambda x: 1 if x == 'Y' else 0)

In [ ]:
# provider claim count
train_combined['ProviderClaimCount'] = train_combined.groupby('Provider')['ClaimID'].transform('count')
test_combined['ProviderClaimCount'] = test_combined.groupby('Provider')['ClaimID'].transform('count')

In [ ]:
# Reimbursement Amount per Chronic Condition
train_combined['ReimbursementPerCondition'] = train_combined['InscClaimAmtReimbursed'] / (train_combined['TotalChronicConditions'] + 1)
test_combined['ReimbursementPerCondition'] = test_combined['InscClaimAmtReimbursed'] / (test_combined['TotalChronicConditions'] + 1)

In [ ]:
# saving files to csv
train_combined.to_csv('train_combined.csv', index=False)
test_combined.to_csv('test_combined.csv', index=False)